<a href="https://colab.research.google.com/github/rg-smith/GIS/blob/main/Exercise3/GIS_exercise3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Class exercise 3: exploring geospatial data with python

This exercise is meant to build your familiarity with some powerful tools for geospatial data analysis and plotting in python. The exercise will mainly work through examples with building complexity. At the end, you will upload your own shapefile and plot it.

The first section only needs to be run once each time you start a new session. It will install the necessary packages, and clone the github repository (copy all the files from my github GIS page to your google colab session).

After running this, if you click the folder icon to the left, you should see a 'GIS' folder. If you don't see it, you can refresh (icon on the top left) and then you should see it.

In [ ]:
!pip install rasterio
!pip install geopandas
!pip install rasterstats
!git clone https://github.com/rg-smith/GIS.git

Now that we have installed the necessary packages, we will import them. Some packages are pre-installed in the version of python running on google colab, so we don't need to install those (but still need to import them).

In [ ]:
import rasterio # tools for working with raster data
import geopandas as gpd #tools for working with vector data
from rasterio.plot import show # rasterio tool for plotting
import pandas as pd #tools for working with tabular (non-spatial) data
import matplotlib.pyplot as plt # tools for generic plotting (non-spatial or spatial)
from rasterstats import zonal_stats # function for estimating zonal statistics
import numpy as np # tools for working with arrays

First, we'll load our raster and shapefile as before.

In [ ]:
raster = rasterio.open('GIS/Rasters/srtm_sw_utah.tif')

In [ ]:
show(raster)

Now, we will explore the data a little bit. This is one of the nice things about python. It's fairly straightforward to dive into the data. We'll plot a histogram of the raster, showing how common each value is.

In [ ]:
plt.figure();plt.hist(raster.read().flatten(),bins=100)

Now we will use the histogram to make some new plots. We can see that most of the data has an elevation below 2000 m. The higher elevations are mountains. Let's say we only want to view parts of the image with a low elevation, below 2000 m. We can use a logical expression to do this. First, we will read our raster as an array, using raster_name.read(band number). Since our raster only has one band (elevation), we just select 1.

In [ ]:
array = raster.read(1)
print(array)

Printing the array shows a summary of the values in the array.

Now, we will plot the array. Note that it will look just like the raster above, but there is no geograhpic information, so the x and y axes are just numbered by rows and column numbers.

In [ ]:
fig, ax = plt.subplots() # create an empty figure
plt.imshow(array) # fill the figure with a plot of the array
plt.colorbar() # add a colorbar

Now, we will view only regions with elevaiton below 2000

In [ ]:
filt = array < 2000
print(filt)

The variable filt is our logical expression. Note how it is also an array, but has changed from numbers to what we call 'boolean', meaning it either says true or false. If we plot this, the True's will be plotted as 1's and False's will be plotted as 0's. **Logical operators include < (less than), > (greater than), and == (equal to). Each of those will return a value of True (1) for the pixels where the statement is true, and False (0) where the statement is false.**

In [ ]:
fig,ax = plt.subplots()
plt.imshow(filt)
plt.colorbar()

Try changing the threshold from 2000 to some other value to more clearly distinguish valleys from mountains. Take a screenshot and include in your lab report.

Now, we will load our shapefile like in the last exercise. We do this using geopandas. We created an alias for geopandas when we imported it (import geopandas as gpd), so to use geopandas functions we type gpd., then the name of the function.

In [ ]:
shp = gpd.read_file('GIS/Shapefiles/parowan_watershed.shp')

Now we've loaded the shapefile, let's take a look at it. The shapefile is loaded as a table, with one column representing the geometry. If we print the shapefile, we can see the table.

In [ ]:
print(shp)

As you can see, there's just one row in this table. Let's plot it to see what it looks like.

In [ ]:
shp.plot()

We can check the coordinate reference systems of raster and crs, as before...

In [ ]:
print('raster coordinate reference system:')
print(raster.crs)
print('shapefile coordinate reference system:')
print(shp.crs)

raster coordinate reference system:
EPSG:4326
shapefile coordinate reference system:
epsg:4326


EPSG is a code that defines a unique coordinate reference system. Google EPSG 4326, and include the details of this crs, including its datum, in your report.

Now that we've verified that these two datasets have the same crs, we can plot them together.

We'll create a blank plot first, then add each dataset to the plot. Note that Rasterio and geopandas have different syntax for plotting. With rasterio we use 'show', and with geopandas we type 'plot()' following a period after the shapefile name.

In [ ]:
fig,ax = plt.subplots()
show(raster,ax=ax)
shp.plot(ax=ax,facecolor='none',edgecolor='red')

Now we will plot our new raster that shows the valleys with the watershed on top. To do this, we need to add some spatial information to the array we created. The spatial information is contained in raster.transform

In [ ]:
transform = raster.transform
print(transform)

Now we can plot the array with the transform, which contains the top-left corner coordinates, as well as the pixel spacing, for the variable 'raster'.

In [ ]:
fig,ax = plt.subplots()
show(filt,transform=transform,ax=ax)
shp.plot(ax=ax,color='none',edgecolor='red')

Take a screenshot and include in your report. What percentage of the watershed would you say is 'valley' vs 'mountain'?

Let's look at the slope.

In [ ]:
slope = np.gradient(array) # calculates the slope in the x and y direction
slope2 = np.max(slope,axis=0) # takes the maximum slope of the x and y directions
fig,ax = plt.subplots()
show(slope2,transform=transform,ax=ax,vmin=0,vmax=30)
shp.plot(ax=ax,color='none',edgecolor='red')

Now try making a filter that identifies valleys based on slope, and plotting it.

## Part 2: Zonal statistics
Now we will use some zonal statistics tools. These work similar to how we used zonal statistics in ArcMap.

The zonal_stats function takes as arguments the shapefile that you want to get zonal statistics over, the raster (needs to be given as an array), and the transform that we defined earlier.

In [ ]:
watershed_elev = pd.DataFrame(zonal_stats(shp,array,affine = transform))
print(watershed_elev)

      min     max         mean  count
0  1735.0  3375.0  2158.817294  26113


/usr/local/lib/python3.7/dist-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


Get a screenshot of the zonal statistics over the watershed. What's the mean, max and min elevation of the watershed?

Now do the same thing, but with the slope2 raster. The transform will be the same as for the other arrays (they all have the same transform).

Now, load your own raster and shapefile. Make sure that they have the same crs (if they don't, do a transform like in exercise 2). Then, do zonal stats with that raster/shapefile combo. Take a screenshot and describe what you did.

## Part 3: Raster algebra

We can do raster algebra here, just like in ArcMap. The units of the raster we are viewing are in meters. Let's convert it to feet. Replace the XXX with a conversion factor, then run the code to convert it to feet.

NOTE: This section is not complete yet. Feel free to do this part, but save your work and don't turn it in yet, because more will be posted next week.

In [ ]:
conversion_factor=XXX
elevation_ft = raster*conversion_factor

Let's look at the average elevation for both rasters to see if it worked.

In [ ]:
print(np.mean(raster))
print(np.mean(elevation_ft))

Take a screenshot of the mean elevation for both rasters. Do you think your conversion worked?

Now we will load a couple MODIS rasters. Load the first one with the code below, then write your own code in the line below to load MODIS_band2.tif, and assign to a variable named modis2.

In [ ]:
modis1 = rasterio.open('GIS/Rasters/MODIS_band1.tif')

Now, we will read the rasters as arrays.

In [ ]:
mod1arr = modis1.read(1)
mod2arr = modis2.read(1)

Now, we will create a new array that is the normalized difference vegetation index (this shows where plants are growing). The formula for the index is (modis band 2 - modis band 1)/(modis band 2 + modis band 1).

Use the arrays you just created to calculate the index. Call the index index (that will ensure the code later on works).

Now let's plot it! We can use show, and the transform from one of the modis rasters (they both have the same transform). First we will set all values above 1 equal to 1, and do the same for -1.

In [ ]:
index[index>1] = 1
index[index<-1] = -1
show(index, modis1.transform)

Now plot the same raster but with the outline of the states. You can check exercise 2 for a refresher of how to do this. Take a screenshot and include with your lab report.